Este Notebook tiene como objetivo naturalizar la columna "docstring" de un dataset orientado a finetuning conversacional para generación de código con LLMs 

# 2 - Naturalización del Dataset

In [1]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import pandas as pd
import torch
from transformers import pipeline
from tqdm import tqdm

# Crear la carpeta /datasets si no existe
output_dir = './datasets'
checkpoint_interval = 50
checkpoint_dir = f'{output_dir}/checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

# Cargar los DataFrames filtrados desde los archivos JSON
df_train_filtered = pd.read_json(f'{output_dir}/train_filtered.json', orient='records', lines=True)
df_validation_filtered = pd.read_json(f'{output_dir}/validation_filtered.json', orient='records', lines=True)
df_test_filtered = pd.read_json(f'{output_dir}/test_filtered.json', orient='records', lines=True)

# Configurar el dispositivo y cargar el pipeline con LLaMA 3 8B Instruct
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

text_generation_pipeline = pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto"
)

# Función para transformar un docstring a una descripción contextual usando el pipeline de LLaMA 3 8B Instruct
def transformar_docstring_con_llama(docstring, code):
    
    messages = [
    {"role": "system", "content": "Imagine you're a user working on a project and you need to request a specific function. Describe your need in a detailed and natural way, as if you were asking a developer to implement this function for you. Do not include any code."},
    {"role": "user", "content": f"I'm working on a project and I need help with the following function:\n\nFunction:\n{code}\n\nCurrent docstring:\n'{docstring}'\n\nPlease describe in a natural way what you need this function to do, including the context or scenario where it will be used."},
    ]

    response = text_generation_pipeline(
        messages,
        max_new_tokens=250,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    
    # Extraer solo la respuesta del "assistant"
    generated_text = ""
    if isinstance(response, list) and isinstance(response[0], dict):
        for part in response[0].get("generated_text", []):
            if part.get("role") == "assistant" and "content" in part:
                generated_text += part["content"] + " "
    
    cleaned_text = generated_text.strip()
    
    print("Cleaned text:", cleaned_text)  # Línea de depuración para ver el texto limpio generado
    return cleaned_text

# Función para procesar y guardar en checkpoints
def procesar_y_guardar(df, df_name):
    contextual_docstrings = []
    start_idx = 0
    
    checkpoint_path = os.path.join(checkpoint_dir, f'{df_name}_contextual_checkpoint.json')
    if os.path.exists(checkpoint_path):
        df_checkpoint = pd.read_json(checkpoint_path, orient='records', lines=True)
        contextual_docstrings = df_checkpoint['docstring_contextual'].tolist()
        start_idx = len(contextual_docstrings)
    
    for i, (_, row) in enumerate(tqdm(df.iterrows(), total=len(df), desc=f"Processing {df_name} dataset")):
        if i < start_idx:
            continue  # Saltar las filas ya procesadas
        
        contextual_docstrings.append(transformar_docstring_con_llama(row['docstring'], row['code']))
        
        if (i + 1) % checkpoint_interval == 0:
            df_checkpoint = pd.DataFrame({'docstring_contextual': contextual_docstrings})
            df_checkpoint.to_json(checkpoint_path, orient='records', lines=True)
            print(f"Checkpoint guardado en {checkpoint_path} hasta la fila {i + 1}.")
    
    df['docstring_contextual'] = contextual_docstrings
    df.to_json(f'{output_dir}/{df_name}_filtered_contextual.json', orient='records', lines=True)
    print(f"{df_name.capitalize()} dataset guardado exitosamente en {output_dir}.")

    if os.path.exists(checkpoint_path):
        os.remove(checkpoint_path)

# Procesar cada dataset con checkpoints
procesar_y_guardar(df_train_filtered, "train")
procesar_y_guardar(df_validation_filtered, "validation")
procesar_y_guardar(df_test_filtered, "test")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Processing train dataset:   0%|          | 0/251820 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\llama\modeling_llama.py:648: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Processing train dataset:   0%|          | 1/251820 [00:07<537:01:09,  7.68s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Cleaned text: I'm building a web scraping tool that extracts data from a specific website. As part of this project, I need to be able to set the text content of certain elements on the webpage. Specifically, I want to be able to set the text content of an element with a specific class.

The function I'm requesting should allow me to set the text content of an element with a given class. For example, if I have an element with the class "heading", I want to be able to set its text content to "New Heading". The function should also allow me to specify the class of the text content, in case there are multiple elements with different classes that need to have their text content set.

In the context of my project, this function will be used to modify the text content of elements on the webpage before extracting the data. This is necessary because the website uses certain elements to display dynamic content that I don't want to include in my data extraction.

I would like the function to be f

Processing train dataset:   0%|          | 2/251820 [00:14<515:16:16,  7.37s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Cleaned text: I'm working on a project that involves processing and analyzing large amounts of text data, specifically using the FoLiA (Format for Language Annotation) format. In this format, each piece of text is represented as an element, which can have various attributes and child elements.

I need a function that allows me to associate a specific document with each element in the FoLiA tree structure. This document contains metadata and additional information about the text data. For example, it might include information about the author, date of creation, or the language in which the text is written.

The function should take an element and a document as input, and then link the element to the document. If the element is part of a larger structure (i.e., it has child elements), the function should recursively apply the same association to all child elements.

Here's an example scenario: let's say I have a FoLiA document that contains a text about a person's life. The document has 

...

Cleaned text: I'm working on a project that involves processing and analyzing linguistic annotations in a specific domain. I have a word-level annotation system where each word is annotated with various types of information, such as part-of-speech tags, named entities, and semantic roles.

I need a function that allows me to retrieve all the span annotations of a specific type that contain a given word. For example, if I have a word "apple" and I want to find all the chunks (a type of annotation) that contain this word, I would use this function to get a list of chunk annotations.

The function should take two arguments: the type of annotation I'm interested in (e.g., "Chunk"), and an optional set constraint (e.g., "food"). This means that if I only care about chunks that are related to food, I can pass "food" as the set constraint.

The function should yield a list of span annotation instances that match the specified type and contain the given word. In the example I provided, the out

Processing train dataset:   0%|          | 24/251820 [02:53<510:28:53,  7.30s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Cleaned text: I'm working on a project that involves building a linguistic analysis tool, specifically a span element that can hold different types of linguistic units such as words, morphemes, and phonemes. 

The function I'm asking for, `setspan`, is used to reset the span element and populate it with new linguistic units. This function should take in any number of arguments, which can be instances of `Word`, `Morpheme`, or `Phoneme`. 

When called, `setspan` should clear out any existing data within the span element and then add the provided linguistic units to the element. This allows me to easily update the span element with new data without having to manually manage the existing data.

For example, if I have a span element that currently contains a word and a morpheme, and I want to add a new phoneme to the span, I would call `setspan` with the word, morpheme, and phoneme as arguments. The function would clear out the existing data and then add the new phoneme to the span, effect

Processing train dataset:   0%|          | 25/251820 [03:01<512:28:14,  7.33s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Cleaned text: I'm working on a project that involves analyzing and processing linguistic data, specifically word references and their relationships with other linguistic elements. The function I'm requesting is a crucial part of this process.

The `_helper_wrefs` function should be an internal helper function that takes two parameters: `targets` and `recurse`. The `targets` parameter is a list that will store the results of the function, and the `recurse` parameter is a boolean that determines whether the function should recursively explore the relationships between linguistic elements.

The function should iterate over each element in the current object (`self`). If an element is a `Word`, `Morpheme`, or `Phoneme`, it should be added to the `targets` list. If an element is a `WordReference`, the function should try to resolve the reference by looking up the corresponding word in the document (`self.doc`) using the `id` attribute. If the reference is resolved, the corresponding word sh